In [ ]:
"""Requests a current Disney auth token.

"""

# https://wiki-wdpro.disney.com/display/Tech/WDPRO+AuthZ+Client+IDs

# stdlib
import time
from os import environ

# library
import requests

IS_PRODUCTION = environ.get("AUTHZ_SOURCE") == "production"

AUTH_URL = f"https://{'' if IS_PRODUCTION else 'stage.'}disneyworld.disney.go.com"
AUTH_URL += "/authentication/get-client-token/"
AUTH_PARAMS = {
    "grant_type": "password",
    "scope": "RETURN_ALL_CLIENT_SCOPES",
    "client_id": f"WDPRT-DISNEYWORLD.GC-{'PROD' if IS_PRODUCTION else 'STAGE'}",
}

class DisneyAuth:
    """Manages Disney authentication credentials."""

    username: str
    password: str

    expires: int = None
    token: str = None
    # swid: str = None

    def __init__(self, username: str = None, password: str = None):
        self.username = username
        self.password = password

    @property
    def expired(self) -> bool:
        """True if a new auth token is needed."""
        if self.expires is None:
            return True
        return time.time() > self.expires

    @property
    def headers(self) -> dict:
        """The auth token as a header dictionary."""
        if self.expired:
            self.update()
        return {"Authorization": "BEARER " + self.token}

    def _update_creds(self):
        """Source Disney auth credentials from the environment or raise an Exception."""
        try:
            if not self.username:
                self.username = environ["dauth_user"]
            if not self.password:
                self.password = environ["dauth_pass"]
        except KeyError:
            raise KeyError("Disney auth credentials not found")

    def update(self):
        """Fetch the client swid and auth token."""
        if self.username is None or self.password is None:
            self._update_creds()
        params = {"username": self.username, "password": self.password}
        params.update(AUTH_PARAMS)
        resp = requests.get(AUTH_URL, params=params).json()
        self.expires = time.time() + resp["expires_in"]
        # self.swid = resp["swid"]
        self.token = resp.get("access_token")